In [10]:
import pandas as pd
df = pd.read_csv('trainstartings.csv')
valid_locations = [
    "TVC", "KCVL", "NCJ", "CAPE", "ERS", "QLN", "ERN", "MDU", "TEN", "SRR",
    "PGT", "CLT", "ED", "TVC", "NCJ", "ERS", "QLN", "ERN", "TEN", "MDU",
    "SRR", "PGT", "TCR", "ALLP", "KTYM", "GUV", "PUU", "KYJ", "CAR"
]
filtered_df = df[df['from'].isin(valid_locations) & df['to'].isin(valid_locations)]
filtered_df.to_csv('trainstartings2.csv', index=False)

The first file contains trainNo,trainName,from,to,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Type. A value is 1 if that train runs on that particular day

The second file contains trainNo,StopNo,StationCode,Day of Journey,Arrival,Departure,Distance

In [3]:
#
import pandas as pd

# Read the CSV file
df = pd.read_csv('alldetails.csv')

# List of valid station codes
valid_station_codes = [
    "TVC", "KCVL", "NCJ", "CAPE", "ERS", "QLN", "ERN", "MDU", "TEN", "SRR",
    "PGT", "CLT", "ED", "TVC", "NCJ", "ERS", "QLN", "ERN", "TEN", "MDU",
    "SRR", "PGT", "TCR", "ALLP", "KTYM", "GUV", "PUU", "KYJ", "CAR"
]

# Filter rows where "StationCo" matches any value in valid_station_codes
filtered_df = df[df['StationCode'].isin(valid_station_codes)]

# Save the filtered rows to a new CSV file
filtered_df.to_csv('alldetails2.csv', index=False)


OVer here we create a weekly timetable of all the trains using the two excel sheets. It contains TrainNo,StartingStationCode,DepartureTime,ArrivingStationCode,ArrivalTime

In [4]:
import pandas as pd

# Load the CSV files
trainstartings_df = pd.read_csv('trainstartings2.csv')
alldetails_df = pd.read_csv('alldetails2.csv')

# Define the days of the week
days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Initialize a list to store edge data
edges_data = []

# Iterate through each unique train
for train_no in alldetails_df['trainNo'].unique():
    # Extract the journey details for this train
    train_data = alldetails_df[alldetails_df['trainNo'] == train_no]

    # Get the starting day info for this train
    train_start_days = trainstartings_df[trainstartings_df['trainNo'] == train_no]

    # Check if train_start_days is not empty
    if not train_start_days.empty:
        # Check each day of the week to find valid start days
        for i, day in enumerate(days_of_week):
            if train_start_days[day].values[0] == 1:  # Train operates on this day
                # Replicate the timetable for this valid starting day
                for j in range(len(train_data) - 1):
                    row1 = train_data.iloc[j]
                    row2 = train_data.iloc[j + 1]

                    # Calculate the correct days of the journey based on the starting day
                    dep_day_of_journey = (i + row1['Day of Journey'] - 1) % 7
                    arr_day_of_journey = (i + row2['Day of Journey'] - 1) % 7

                    # Format the departure and arrival times with the correct day prefix
                    departure_time = f"{days_of_week[dep_day_of_journey][:3]} {row1['Departure']}"
                    arrival_time = f"{days_of_week[arr_day_of_journey][:3]} {row2['Arrival']}"

                    # Append the edge data
                    edges_data.append([
                        train_no, row1['StationCode'], departure_time,
                        row2['StationCode'], arrival_time
                    ])

# Create a DataFrame with the collected edge data
edges_df = pd.DataFrame(edges_data, columns=[
    'TrainNo', 'StartingStationCode', 'DepartureTime',
    'ArrivingStationCode', 'ArrivalTime'
])

# Save the result to a CSV file
edges_df.to_csv('train_edges_repeated_days.csv', index=False)

print("The timetable with repeated days has been processed and saved as 'train_edges_repeated_days.csv'.")


The timetable with repeated days has been processed and saved as 'train_edges_repeated_days.csv'.


So now I am basically adding arrival time - departure time for all trains. I have converted everything to minutes and then divided by 60 at the end. We get some hours which we will divide by 40.

In [9]:
import pandas as pd
from datetime import datetime

# Specify the file path of your CSV file
file_path = 'train_edges_repeated_days.csv'

# Read the CSV file
df = pd.read_csv(file_path)

# Function to convert day and time into total minutes
def time_to_minutes(day_str, time_str):
    day_str = day_str.strip().lower()
    days_offset = {'sun': 0, 'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5, 'sat': 6}
    
    time = datetime.strptime(time_str, "%H:%M")
    minutes = time.hour * 60 + time.minute
    
    return days_offset[day_str[:3]] * 1440 + minutes 

# Initialize total time in minutes
total_time_minutes = 0

# Loop through the rows to accumulate total time
for index, row in df.iterrows():
    dep_time = time_to_minutes(row[2].split()[0], row[2].split()[1])  # Departure time in minutes
    arr_time = time_to_minutes(row[4].split()[0], row[4].split()[1])  # Arrival time in minutes
    
    # Add the time difference to total time
    # total_time_minutes += arr_time - dep_time
    if(arr_time<dep_time):
        total_time_minutes+=10080-dep_time+arr_time
    else:
        total_time_minutes += arr_time - dep_time
    print(arr_time," ",dep_time," ",arr_time-dep_time,"\n")

# Convert total time to hours
total_time_hours = total_time_minutes / 60

# Print the total time in hours
print(f"Total time: {total_time_hours:.2f} hours")

1111   1055   56 

1170   1112   58 

2551   2495   56 

2610   2552   58 

3991   3935   56 

4050   3992   58 

5431   5375   56 

5490   5432   58 

6871   6815   56 

6930   6872   58 

8311   8255   56 

8370   8312   58 

9751   9695   56 

9810   9752   58 

285   225   60 

350   286   64 

1725   1665   60 

1790   1726   64 

3165   3105   60 

3230   3166   64 

4605   4545   60 

4670   4606   64 

6045   5985   60 

6110   6046   64 

7485   7425   60 

7550   7486   64 

8925   8865   60 

8990   8926   64 

1050   960   90 

2490   2400   90 

3930   3840   90 

5370   5280   90 

6810   6720   90 

8250   8160   90 

9690   9600   90 

540   445   95 

1980   1885   95 

3420   3325   95 

4860   4765   95 

6300   6205   95 

7740   7645   95 

9180   9085   95 

319   270   49 

455   321   134 

465   457   8 

1759   1710   49 

1895   1761   134 

1905   1897   8 

3199   3150   49 

3335   3201   134 

3345   3337   8 

4639   4590   49 

4775   4641   134 

4785 